In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Load the MNIST dataset

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

In [ ]:
mnist = fetch_openml("mnist_784", version=1)

In [ ]:
mnist.keys()

In [ ]:
X, y = mnist["data"], mnist["target"]
# X is a pd.DataFrame, but we need a numpy array
X = X.to_numpy()
y = y.to_numpy().astype(np.uint8)

In [ ]:
print(type(X))
print(type(y))
print(X.shape)
print(y.shape)

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
import numpy as np

sample = np.random.randint(0, X.shape[0] - 1)
some_digit = X[sample]
some_digit_reshape = some_digit.reshape(
    tuple(
        [np.sqrt(X.shape[1]).astype("int")]*2
    )
)


In [ ]:
plt.imshow(some_digit_reshape, cmap="binary")

In [ ]:
y[sample]

## Split train and test sets

In [ ]:
train_test_ratio = 0.8
limit = round(X.shape[0]*train_test_ratio)
# the dataset is already shuffled
X_train, X_test, y_train, y_test = X[:limit], X[limit:], y[:limit], y[limit:]

In [ ]:
# TODO: some data exploration to check categories are correctly shuffled
import seaborn as sns

sns.displot(y_train)
sns.displot(y_test)

# Binary classifier

In [ ]:
# transform target vector
y_train_5 = y_train == 5
y_test_5 = y_test == 5

## Stocastic gradient descent classifier (sklearn)

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
# cross validation from scratch
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3)

In [ ]:
for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    print(f"train_index: {train_index}")
    print(f"test_index: {test_index}")
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(f"fold accuracy: {n_correct / len(y_pred)}")

In [ ]:
# sklearn built-in function for cross validation
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

In [ ]:
# create a dummy classifies

from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y): 
        pass
    def predict(self, X): 
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
never5classifier = Never5Classifier()

cross_val_score(never5classifier, X_train, y_train_5, cv=3, scoring="accuracy")

In [ ]:
np.mean(y == 5)

## Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
# cross_val_predict is similat to cross_val_score, but it returns 
# predicted values in each test fold

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
# lets create first confusion matrix by hand

positives_index = np.flatnonzero(y_train_pred)
negatives_index = np.flatnonzero(~y_train_pred)
TP = sum(y_train_5[positives_index])
FP = len(positives_index) - TP
TN = sum(~y_train_5[negatives_index])
FN = len(negatives_index) - TN

In [ ]:
conf_matrix = np.array([[TN, FP], [FN, TP]])

In [ ]:
# with sklearn 
from sklearn.metrics import confusion_matrix
bi_conf_matrix = confusion_matrix(y_train_5, y_train_pred)

In [ ]:
print(conf_matrix)
print(bi_conf_matrix)

In [ ]:
# let us compute precision and recall by hand and with sklearn built-in functions
from sklearn.metrics import precision_score, recall_score
precision = TP / (TP + FP)
bi_precision = precision_score(y_train_5, y_train_pred)
recall = TP / (TP + FN)
bi_recall = recall_score(y_train_5, y_train_pred)

In [ ]:
print(f"Hand made precision: {precision:0.2f}")
print(f"Built in precision: {bi_precision:0.2f}")
print(f"Hand made recall: {recall:0.2f}")
print(f"Built in recall: {bi_recall:0.2f}")

In [ ]:
# now f1 score
from sklearn.metrics import f1_score
f1 = 2 / (1/precision + 1/recall)
bi_f1 = f1_score(y_train_5, y_train_pred)
print(f"Hand made F1 score: {f1:0.2f}")
print(f"Built in F1 score: {bi_f1:0.2f}")

## The precision/recall trade off

In sklearn, classifiers have a method called `decision_function` that gives us the score a certain instance has. We classify our inputs depending of this score being higher than a certain therhold. For example, the `SGDClassifier` uses a threshold equal to 0. 

In [ ]:
y_train_scores = sgd_clf.decision_function(X_train)

In [ ]:
y_train_scores

In [ ]:
threshold = 0
y_train_pred_2 = (y_train_scores > threshold)

In [ ]:
y_train_pred = sgd_clf.predict(X_train)

In [ ]:
all(y_train_pred == y_train_pred_2)

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds, point_precision=None):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    if (point_precision):
        point_recall = recalls[np.argmax(precisions >= point_precision)]
        point_threshold = thresholds[np.argmax(precisions >= point_precision)]
        plt.plot([point_threshold, point_threshold], [0., point_precision], "r:")                 
        plt.plot([-50000, point_threshold], [point_precision, point_precision], "r:")                                
        plt.plot([-50000, point_threshold], [point_recall, point_recall], "r:")
        plt.plot([point_threshold], [point_precision], "ro")                                             
        plt.plot([point_threshold], [point_recall], "ro")
    plt.legend(loc="center right", fontsize=16) 
    plt.xlabel("Threshold", fontsize=16)        
    plt.grid(True)                              
    plt.axis([-50000, 50000, 0, 1]) 
    plt.title("Precision/Recall curve")            

In [ ]:
plt.figure(figsize=(8, 4))    
plot_precision_recall_vs_threshold(precisions, recalls, thresholds, point_precision=0.8)                            
save_fig("precision_recall_curve")
plt.show()

In [ ]:
def plot_precision_vs_recall(precisions, recalls, point_precision=None):
    plt.plot(precisions, recalls, "b--", linewidth=2)
    if (point_precision):
        point_index = np.argmin(np.abs(precisions - 0.8))
        point_precision = precisions[point_index]
        point_recall = recalls[point_index]
        plt.plot([point_precision, point_precision], [0, point_recall], "r:", linewidth=2)
        plt.plot([0, point_precision], [point_recall, point_recall], "r:", linewidth=2)
        plt.plot([point_precision], [point_recall], "ro")
    plt.xlabel("Precision")
    plt.ylabel("Recall")
    plt.grid(True)
    plt.title("Precision vs Recall")

In [ ]:
plt.figure(figsize=(8,4))
plot_precision_vs_recall(precisions, recalls, 0.8)
save_fig("precision_recall")
plt.show()

## ROC curve
Another common tool when using binary classifiers is the so called **ROC curve** *(receiver operating characteristic)* which plots TPR vs FPR. 
- TPR (true positive rate): also called sensitivity or recall, is the ratio of real positives that have been correctly classified
$$
TPR = \frac{TP}{TP + FN}
$$
- FPR (false positive rate): is the ratio of real negatives that has been incorrectly classified as positives
$$
FPR = \frac{FP}{FP + TN} = 1 - \frac{TN}{FP + TN} = 1 - \textit{specificity}
$$

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thersholds = roc_curve(y_train_5, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.grid()
    plt.title("ROC curve")

plt.figure(figsize=(8,4))
plot_roc_curve(fpr, tpr)
plt.show()

### AUC score
One way to evaluate classifiers is to look a the AUC (area under the curve). The higher the area, the best a classifier will performe. Sklearn provides a function to compute the ROC AUC, `roc_auc_score`

In [ ]:
from sklearn.metrics import roc_auc_score
sgd_auc = roc_auc_score(y_train_5, y_scores)
print(sgd_auc)

## ROC Curve vs PR Curve
The key point when evaluating a classifier is to decide which is more important, false positives or false negatives. For example,
- Predict cancer on patients: minimize false negatives.
- Detect safe videos for children: minimize false positives.
> You should use PR Curve whenever the positive class is rare or when you care more about the false positives than the false negatives. Otherwise, use the ROC curve and de AUC score. 

In the detect 5 digits problem, looking at the AUC we could conclude there is no room for improvement (because there are few 5s (positives)), but the PR curve makes it clear we can do better. 

Now let us train another classifier (in this case a random forest classifier) and compare its ROC curve with the Stocastic Gradient Descend Classifier. 

`RandomForestClassifier` does not have a `decision_function` method like `SGDClassifier`. Instead, it has a `predict_proba` method. Sklearn classifiers generally have one or the other, or both. The `predict_proba` method returns an array containing a row per instance and a column per class, each containing the probability that the fiven instance belongs to the given class. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, method="predict_proba")

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # probability of positive class
fpr_forest, tpr_forest, thersholds_forest = roc_curve(y_train_5, y_scores_forest)
precision_forest, recall_forest, thersholds_forest = precision_recall_curve(y_train_5, y_scores_forest)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(fpr_forest, tpr_forest, linewidth=2, label="Random Forest")
plot_roc_curve(fpr, tpr, label="SGD")
plt.legend()
plt.show()

In [ ]:
forest_auc = roc_auc_score(y_train_5, y_scores_forest)

In [ ]:
# precision/recall of Random Forest Classifier
y_pred_forest = y_probas_forest[:, 1] > 0.5
forest_precision = precision_score(y_train_5, y_pred_forest)
forest_recall = recall_score(y_train_5, y_pred_forest)
sgd_precision = precision
sgd_recall = recall

In [ ]:
import pandas as pd

metrics_data = {
    "Forest": [forest_auc, forest_precision, forest_recall], 
    "SGD": [sgd_auc, sgd_precision, sgd_recall]
}

df_summary = pd.DataFrame(
    metrics_data, index=["auc", "precision", "recall"]
)

In [ ]:
# format pandas table

def make_pretty(styler):
    styler.set_caption("Metric comparison between classfiers")
    styler.background_gradient(axis=None, vmin=0, vmax=1, cmap="YlGnBu")
    return styler

In [ ]:
df_summary.style.pipe(make_pretty)

## Multiclass clasification

Some algorithms (such as Stocastic Gradient Descend, Random Forest or Naive Bayes classifiers) are capable of handling multiple classes natively. Others, like Logistic Regression or Support Vector Machine classifiers are stricly binary, but we can turn them into multiclass classifiers with the following strategies: 
1. *one-versus-all* or *one-versus-the-rest* (OvR): train a binary classifier for each class and predict the class whose score is maximum. 
2. *one-versus-one* (OvO): train a binary classifier for each pair of classes, that is, $N \times (N - 1) / 2$ classifiers and predict the class which wins the most duels. You need to train more models, but the training sets get smaller since you only need to retain two classes from the whole dataset. This is convenient for classifier that scale poorly with the size of the triainig set, such as Support Vector Machines. Otherwise, OvR is preferred.  

Scikit-Learn detects when a binary classifier is used for multiclass detection and automatically runs OvR or OvO depeding on the the lgorithm. 

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
svm_clf.predict([some_digit])

In [ ]:
y_train[sample]

Under the hood, sklearn trained 45 classifiers with a OvO strategy. Let us have a look at the `decision_function` method 

In [ ]:
some_digit_scores = svm_clf.decision_function([some_digit])

In [ ]:
some_digit_scores

In [ ]:
np.argmax(some_digit_scores)

In [ ]:
svm_clf.classes_

In this case, the `classes_` atribute matches the digit they represent, but this is not going to happend in general

In [ ]:
# use ovr stretegy
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, y_train)

In [ ]:
ovr_clf.predict([some_digit])

In [ ]:
from sklearn.linear_model import SGDClassifier
# SGD can predict multiple classes
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)

In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
from skelearn.preprocessing import StandardScaler 
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train)

In [ ]:
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

### Evaluating a multiclass classifier


In [ ]:
conf_mx = confusion_matrix(y_train, y_train_pred)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

## Multilabel classification
Sometimes we want to predict not only one class, but many of them. Let us take a look to a quick example

In [32]:
from sklearn.neighbors import KNeighborsClassifier
y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2)
knn_clf = KNeighborsClassifier()

In [19]:
y_multilabel = np.c_[y_train_large, y_train_odd]

In [31]:
y_multilabel.shape

(56000, 2)

In [33]:
knn_clf.fit(X_train, y_multilabel)

KNeighborsClassifier()

In [34]:
knn_clf.predict([some_digit])

array([[0, 0]], dtype=uint8)

In [36]:
y[sample]

2